Start with Shrinkage models:

In [36]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoLarsCV
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np


In [28]:
data = pd.read_csv('data/price_data_filtered.csv', index_col=0, parse_dates=True)
data.tail()

,STOXX EUROPE 600 \nE - PRICE INDEX,STOXX EUROPE 600 \nBANKS E - PRICE INDEX,STOXX EUROPE 600 \nFINANCIALS E - PRICE INDEX,STOXX EUROPE 600 \nAUTO & PARTS E - PRICE INDEX,STOXX EUROPE 600 \nINSURANCE E - PRICE INDEX,STOXX EUROPE 600 \nTECHNOLOGY E - PRICE INDEX,STOXX EUROPE 600 \nUTILITIES E - PRICE INDEX,STOXX EUROPE 600 \nCHEMICALS E - PRICE INDEX,STOXX EUROPE 600 \nBASIC RESOURCE E - PRICE INDEX,STOXX EUROPE 600 \nCON & MAT E - PRICE INDEX,STOXX EUROPE 600 \nHEALTH CARE E - PRICE INDEX,STOXX EUROPE 600 \nINDS GDS & SVS E - PRICE INDEX,STOXX EUROPE 600 \nMEDIA E - PRICE INDEX,STOXX EUROPE 600 \nTELECOM E - PRICE INDEX
2021-09-01,473.123,137.105,218.459,625.259,309.675,816.033,396.884,1312.686,593.656,631.365,1040.949,779.914,360.955,240.289
2021-10-01,452.897,139.835,215.834,630.213,303.691,741.713,363.426,1248.539,540.547,581.521,986.327,737.698,351.207,228.949
2021-11-01,478.865,150.619,231.872,678.609,320.779,807.080,392.884,1316.179,564.657,617.930,1056.962,762.544,356.543,223.070
2021-12-01,470.862,139.734,220.570,656.661,310.670,812.144,389.468,1306.550,570.064,611.488,1030.112,760.739,359.789,225.157
2022-01-01,489.988,146.388,229.872,675.257,325.112,806.015,407.840,1368.250,601.215,647.029,1076.658,802.949,372.238,231.135


In [40]:
X = np.asarray(data.index.values.reshape(-1, 1))
y = data.iloc[:, 0].values


#X = data.iloc[:, 0].values.reshape(-1, 1)
#y = data.iloc[:, 1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
#print(ridge.score(X_test, y_test))
ridge_pred = ridge.predict(X_test)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>)

In [38]:
X 


array(['1987-01-01T00:00:00.000000000', '1987-02-01T00:00:00.000000000',
       '1987-03-01T00:00:00.000000000', '1987-04-01T00:00:00.000000000',
       '1987-05-01T00:00:00.000000000', '1987-06-01T00:00:00.000000000',
       '1987-07-01T00:00:00.000000000', '1987-08-01T00:00:00.000000000',
       '1987-09-01T00:00:00.000000000', '1987-10-01T00:00:00.000000000',
       '1987-11-01T00:00:00.000000000', '1987-12-01T00:00:00.000000000',
       '1988-01-01T00:00:00.000000000', '1988-02-01T00:00:00.000000000',
       '1988-03-01T00:00:00.000000000', '1988-04-01T00:00:00.000000000',
       '1988-05-01T00:00:00.000000000', '1988-06-01T00:00:00.000000000',
       '1988-07-01T00:00:00.000000000', '1988-08-01T00:00:00.000000000',
       '1988-09-01T00:00:00.000000000', '1988-10-01T00:00:00.000000000',
       '1988-11-01T00:00:00.000000000', '1988-12-01T00:00:00.000000000',
       '1989-01-01T00:00:00.000000000', '1989-02-01T00:00:00.000000000',
       '1989-03-01T00:00:00.000000000', '1989-04-01